In [ ]:
import pandas as pd
import csv
import matplotlib as plt
import sqlite3


## Creating pandas dataframes from .csvs

In [ ]:
Full_df = pd.read_csv('./VC62_Moth_Recording_PM.csv')
# Full_df.head(10)

Full_columns = list(Full_df.keys())

print("Columns are: ")
print(Full_columns)



In [ ]:
# These "raw" data spreadsheets need tidying.
# They can also be joined together to a master dataframe.
Data_2021 = pd.read_csv('./2021_raw.csv')
Data_2022 = pd.read_csv('./2022_raw.csv')
Data_2023 = pd.read_csv('./2023_raw.csv')

raw_data = {2021:Data_2021, 2022:Data_2022, 2023:Data_2023}

# Data_2021.head()

## Analysing Pandas dataframes

In [ ]:
Full_df.head(20)


In [ ]:
print(f"Number of entries in dataframe is {len(Full_df)}")

Taxon = Full_df["Taxon"]
Taxon_set = set(Taxon)

print(f"Number of unique Taxons is {len(Taxon_set)}")

Dates = Full_df["Date"]
Dates_set = set(Dates)

print(f"Number of unique Dates is {len(Dates_set)}")

print("Unique Taxons are:")
print(Taxon_set)

In [ ]:
def Get_mean_taxon_sightings(df, Taxon):
    """
    Given string of the moth taxon, returns the total number of times that taxon was seen divided by the total number of unique dates (i.e the attempts at sighting something).
    """
    indices_of_instances = df["Taxon"][df["Taxon"]==Taxon].index.tolist()
    print(f"{Taxon} was seen on {len(indices_of_instances)} separate instance(s).")

    Num_times_per_instance = Full_df["Quantity"][test_indices].values
    Total_times = sum(Num_times_per_instance)
    print(f"{Taxon} was seen a total of {Total_times} time(s).")

    Mean_when_seen = Total_times/len(indices_of_instances)
    print(f"When {Taxon} was seen, it was seen a mean of {Mean_when_seen} times.")

    Dates = df["Date"]
    Dates_set = set(Dates)
    Total_unique_dates = len(Dates_set)

    Mean_sightings = Total_times/Total_unique_dates
    print(f"Mean number of {Taxon} sightings overall was {Mean_sightings}.")

    return Mean_sightings


In [ ]:
Get_mean_taxon_sightings(Full_df, 'Lacanobia thalassina')

## Creating SQL (sqlite3) database from dataframe

In [ ]:
class DatabaseManager:
    ''' Database Manager '''

    def __init__(self, db_name):
        self.db_name = db_name  # database name
        self.conn = None        # connection


    def check_database(self):
        ''' Check if the database exists or not '''
        try:
            print(f'Checking if {self.db_name} exists or not...')
            self.conn = sqlite3.connect(f"file:{self.db_name}?mode=rw", uri=True) #Used https://stackoverflow.com/questions/12932607/how-to-check-if-a-sqlite3-database-exists-in-python
            print(f'Database exists. Succesfully connected to {self.db_name}')
            
        except sqlite3.OperationalError as err:

            print(f'Database named {self.db_name} does NOT exist')

            print(err)

    def close_connection(self):
        ''' Close connection to database '''

        if self.conn is not None:
            self.conn.close()


In [ ]:
dbmngr = DatabaseManager("moth_database.db")
dbmngr.check_database()
dbmngr.close_connection()


# con = sqlite3.connect('file:aaa.db?mode=rw', uri=True)

In [ ]:
create_db_from_df=input("Load in data from pandas dataframe? y/n")

if create_db_from_df == "y": 
    con = sqlite3.connect("moth_database.db")
    cur = con.cursor()
    Full_df.to_sql('moths', con, if_exists='append', index = False)
    print("Produced database using pandas dataframe.")

else: 
    print("Loaded database already saved as \"moth_database.db\".")
    con = sqlite3.connect("moth_database.db")
    cur = con.cursor()


In [ ]:
res = cur.execute("SELECT * FROM moths") #Querying full database
# res.fetchall()

full_data = cur.execute("SELECT * FROM moths").fetchall()
print(type(full_data))
print(full_data)

print(len(full_data))


In [ ]:
colnames = cur.description
colnames_list = []
for row in colnames:
    # print(row[0])
    colnames_list.append(row[0])

print("Database columns are: ")
print(colnames_list)

# names = list(map(lambda x: x[0], cursor.description))

## Manipulating SQL database

In [ ]:
# cur.execute(f"CREATE TABLE moths{Full_columns}") #Creating a table


## Running SQL scripts

In [ ]:
# Running .SQL scripts with sqlite3 package
